# 1. Extracting Data from OpenWeatherMap API

In [4]:
# Import the dependencies used in the project
import numpy as np
from citipy import citipy
from config import weather_key
import requests
import pandas as pd

### 1.1 Generating a list of cities

In [2]:
# Create a range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create two empty lists for holding the latitude/longitude and city names

lats_lngs = []
city_names = []

# Create a set of random co-ordinates to fill the lat_long list
lats = np.random.uniform(lat_range[0], lat_range[1], size = 1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size = 1500)
lats_lngs = zip(lats, lngs)

# Identify the nearest city for each generated co-ordinate
for lat_lng in lats_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the located city is not in the current list, add to the city_names list
    if city not in city_names:
        city_names.append(city)
        

city_names = sorted(city_names)

# Print the number of cities found with the code
print(f'From 1,500 co-ordinates, a total of {len(city_names)} cities were found ({(len(city_names) / 1500) * 100:.2f}% retrived)')
print(f'Data will be collected from {city_names[0]} to {city_names[-1]}!')

From 1,500 co-ordinates, a total of 612 cities were found (40.80% retrived)
Data will be collected from abonnema to zhigansk!


### 1.2 Performing the API Calls

In [3]:
# Call the API's url, reset the counters for the loop and create a variable to store the city's weather data
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_key
record_counter = 1
set_counter = 1
city_data = []


# Create a loop for scraping data from the API
for x, city in enumerate(city_names):
    city_url = url + "&q=" + city
    
    # Run an API request for each city in the cities list
    try:
        city_weather = requests.get(city_url).json()
        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]
        
        # Append the city information into the city_data list
        city_data.append({"City": city,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
        record_counter = record_counter + 1
    except:
        record_counter = record_counter + 1
        pass

print(f'The API was available to find the weather information for {len(city_data)} cities in total, {(len(city_data) / len(city_names)) * 100:.2f}% of the total city names generated!')

The API was available to find the weather information for 560 cities in total, 91.50% of the total city names generated


### 1.3 Create a DataFrame and Save as .csv

In [5]:
city_df = pd.DataFrame(city_data)
city_df.to_csv("Data/City Data.csv", index = False)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,abonnema,4.7084,6.7931,75.49,87,100,8.21,NG,1664127579
1,abu dhabi,24.4667,54.3667,89.71,81,0,4.61,AE,1664127421
2,acurenam,1.0322,10.6488,69.08,99,99,3.83,GQ,1664127579
3,adrar,20.5022,-10.0711,99.12,22,24,7.70,MR,1664127579
4,ahipara,-35.1667,173.1667,57.38,89,92,10.49,NZ,1664127580
